# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd

In [178]:
df = pd.read_csv('previsao_de_renda.csv')

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [291]:
import numpy as np

import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


import matplotlib.pyplot as plt
import seaborn as sns


In [221]:
df_linha = df.iloc[:,3:].copy()

df_linha = pd.get_dummies(df_linha)
df_linha.columns = [c.replace(' ','_') for c in df_linha.columns]
df_linha = df_linha.astype('float64')
df_linha.fillna(0, inplace=True)


X = df_linha.iloc[:,:-1].copy()
y = df_linha.iloc[:,-1].copy()


n = int(.75*1500)

df_linha_train = df_linha[:-n]
df_linha_test = df_linha[-n:]

#X_train , y_train = X[:-n] , y[:-n]
#X_test , y_test = X[-n:] , y[-n:]

In [223]:
y_train, X_train = patsy.dmatrices('renda ~ posse_de_veiculo + posse_de_imovel + qtd_filhos + idade +\
                            tempo_emprego + qt_pessoas_residencia + sexo_F + sexo_M + tipo_renda_Assalariado +\
                            tipo_renda_Bolsista + tipo_renda_Empresário + tipo_renda_Pensionista +\
                            tipo_renda_Servidor_público + educacao_Primário + educacao_Pós_graduação +\
                            educacao_Secundário + educacao_Superior_completo + educacao_Superior_incompleto +\
                            estado_civil_Casado + estado_civil_Separado + estado_civil_Solteiro + estado_civil_União +\
                            estado_civil_Viúvo + tipo_residencia_Aluguel + tipo_residencia_Casa + tipo_residencia_Com_os_pais +\
                            tipo_residencia_Comunitário + tipo_residencia_Estúdio', 
                                   data = df_linha_train)

In [150]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Ex. 2

In [191]:
r_score=[]

for alpha in alphas:

    md = sm.OLS(y_train, X_train)

    reg = md.fit_regularized(method = 'elastic_net',
                             refit = True,
                             L1_wt = 0.001,
                             alpha = alpha)

    r_score.append(reg.rsquared)

r_score

[0.26369485613025634,
 0.26369485613025634,
 0.26369485613025634,
 0.26369485613025634,
 0.26369485613025634,
 0.26369485613025634]

# Ex. 3

In [192]:
r_score=[]

for alpha in alphas:
    
    md = sm.OLS(y_train, X_train)

    reg = md.fit_regularized(method = 'elastic_net',
                             refit = True,
                             L1_wt = 1,
                             alpha = alpha)

    r_score.append(reg.rsquared)

r_score

[0.26369485613025634,
 0.26369485613025634,
 0.26369485613025634,
 0.26369485613025634,
 0.26369485613025634,
 0.26369485613025634]


O MÉTODO LASSO E RIDGE APRESENTAM OS MESMOS VALORES DE R2 

# Ex. 4

In [224]:
X_train, X_test, y_train, y_test = train_test_split(X,  y, test_size= 0.25 )

In [225]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [226]:
variaveis = stepwise_selection(X, y)

print('resulting features:')
print(variaveis)

Add  tipo_residencia_Casa           with p-value 0.0
#############
['tipo_residencia_Casa']
Add  tipo_residencia_Com_os_pais    with p-value 0.0
#############
['tipo_residencia_Casa', 'tipo_residencia_Com_os_pais']
Add  tipo_residencia_Aluguel        with p-value 0.0
#############
['tipo_residencia_Casa', 'tipo_residencia_Com_os_pais', 'tipo_residencia_Aluguel']
Add  tipo_residencia_Estúdio        with p-value 0.0
#############
['tipo_residencia_Casa', 'tipo_residencia_Com_os_pais', 'tipo_residencia_Aluguel', 'tipo_residencia_Estúdio']
Add  tipo_residencia_Comunitário    with p-value 0.0
#############
['tipo_residencia_Casa', 'tipo_residencia_Com_os_pais', 'tipo_residencia_Aluguel', 'tipo_residencia_Estúdio', 'tipo_residencia_Comunitário']
Add  tempo_emprego                  with p-value 0.0
#############
['tipo_residencia_Casa', 'tipo_residencia_Com_os_pais', 'tipo_residencia_Aluguel', 'tipo_residencia_Estúdio', 'tipo_residencia_Comunitário', 'tempo_emprego']
Add  renda               

In [253]:
reg_2 = smf.ols('renda ~ tipo_residencia_Casa + tipo_residencia_Com_os_pais +\
               tipo_residencia_Aluguel + tipo_residencia_Estúdio + tipo_residencia_Comunitário +\
               tempo_emprego', data = df_linha_train).fit()

reg_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.152
Model:                            OLS   Adj. R-squared:                  0.151
Method:                 Least Squares   F-statistic:                     413.5
Date:                Sat, 01 Apr 2023   Prob (F-statistic):               0.00
Time:                        18:21:06   Log-Likelihood:            -1.4372e+05
No. Observations:               13875   AIC:                         2.875e+05
Df Residuals:                   13868   BIC:                         2.875e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                    3020.5995    379.747      7.954      0.000    2276.244    3764.955
tipo_residencia_Casa         -374.4180    380.610     -0.984      0.325   -1120.465     371.629
tipo_residencia_Com_os_pais  -696.9410    485.626     -1.435      0.151   -1648.834     254.952
tipo_residencia_Aluguel       652.3866    677.081      0.964      0.335    -674.784    1979.557
tipo_residencia_Estúdio      2158.7360    936.513      2.305      0.021     323.043    3994.429
tipo_residencia_Comunitário -1043.7371   1046.094     -0.998      0.318   -3094.222    1006.748
tempo_emprego                 473.9569      9.560     49.575      0.000     455.217     492.696
==============================================================================
Omnibus:                    19841.706   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         11890081.484
Skew:                           8.298   Prob(JB):                         0.00
Kurtosis:                     145.447   Cond. No.                         170.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Ex. 5

Os modelos do exercício 3 parecem ser melhores por apresentarem um R2 maior

# Ex. 6

In [ ]:
#y_train, X_train = patsy.dmatrices('np.power(renda,.5) ~ posse_de_imovel + idade + tempo_emprego +\
#                                    sexo_M + tipo_renda_Pensionista +\
#                                    estado_civil_Casado', data = df_linha_train)

In [295]:
y_train, X_train = patsy.dmatrices('np.log(renda) ~ posse_de_imovel + idade + tempo_emprego +\
                                    sexo_M + tipo_renda_Pensionista', data = df_linha_train)

reg_1 = sm.OLS(y_train, X_train).fit()

reg_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.339
Model:                            OLS   Adj. R-squared:                  0.339
Method:                 Least Squares   F-statistic:                     1424.
Date:                Sat, 01 Apr 2023   Prob (F-statistic):               0.00
Time:                        18:37:52   Log-Likelihood:                -15110.
No. Observations:               13875   AIC:                         3.023e+04
Df Residuals:                   13869   BIC:                         3.028e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  7.2880      0.031    235.223      0.000       7.227       7.349
posse_de_imovel            0.0909      0.013      6.900      0.000       0.065       0.117
idade                      0.0039      0.001      5.241      0.000       0.002       0.005
tempo_emprego              0.0612      0.001     58.213      0.000       0.059       0.063
sexo_M                     0.8043      0.013     59.799      0.000       0.778       0.831
tipo_renda_Pensionista     0.2277      0.024      9.411      0.000       0.180       0.275
==============================================================================
Omnibus:                        1.092   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.579   Jarque-Bera (JB):                1.064
Skew:                           0.011   Prob(JB):                        0.588
Kurtosis:                       3.037   Cond. No.                         254.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Ex. 7

In [296]:
X_train, X_test, y_train, y_test = train_test_split(X,  y, test_size= 0.25 )

In [305]:
reg_tree = DecisionTreeRegressor()
reg_tree = reg_tree.fit(X_train,y_train)

reg_tree..score(X_test,y_test)

1.0

In [308]:
mean_squared_error(y_test,reg_tree.predict(X_test))

0.0